### 문제 1-2: 2단계 체인 만들기 - 영화 추천 시스템 해답

## 환경 설정

In [1]:
# 필요한 라이브러리 설치
#%pip install -q langchain langchain-openai python-dotenv

In [2]:
# 환경 변수 설정
from dotenv import load_dotenv
import os

# .env 파일에서 API 키 로드
load_dotenv()

True

In [3]:
# 필요한 라이브러리 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

## 문제 2 해답 구현

### 요구사항
1. 1단계 체인: 장르를 입력받아 영화 1편 추천
2. 2단계 체인: 추천받은 영화의 3줄 줄거리 요약 제공
3. ChatPromptTemplate 사용
4. 두 체인을 LCEL로 연결
5. 각 단계의 결과를 모두 출력하여 과정 확인

In [4]:
# 1단계: 장르별 영화 추천 프롬프트
movie_recommendation_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 전문가입니다. 사용자가 요청한 장르에 맞는 영화 1편을 추천하고 간단한 설명을 제공해주세요."),
    ("user", "{genre} 장르의 한국영화 1편을 추천해주세요. 영화 제목과 왜 이 영화를 추천하는지 이유도 함께 알려주세요.")
])

In [5]:
# 2단계: 영화 줄거리 요약 프롬프트
plot_summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 줄거리 요약 전문가입니다. 영화의 핵심 내용을 3줄로 간결하게 요약해주세요."),
    ("user", "다음 영화 추천 내용을 바탕으로 해당 영화의 등장인물과 줄거리를 정확히 10줄로 요약해주세요: {movie}")
])

In [6]:
# LLM 모델 초기화
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# 1단계 체인: 영화 추천
recommendation_chain = movie_recommendation_prompt | llm | StrOutputParser()
# Step 1: 사용자가 입력한 장르에 따라 영화 추천
movie = recommendation_chain.invoke({"genre": "Drama"})  # 영화 제목 얻기
print(" 추천된 영화:", movie)  # movie 값 출력

# 2단계 체인: 줄거리 요약 (1단계 결과를 입력으로 받음)
plot_chain = (
    {"movie": recommendation_chain}  # 1단계 체인의 결과를 movie_info로 전달
    | plot_summary_prompt
    | llm
    | StrOutputParser()
)

 추천된 영화: Drama 장르의 한국영화 1편으로 '올드보이' (2003)를 추천합니다. '올드보이'는 '박찬욱' 감독이 연출한 영화로, 주인공 서이채 (최민식)가 15년 동안 감금된 후, 자신의 과거와 비밀을 풀어나가는 과정을 그린 작품입니다. 

이 영화는 인간의 심오한 심리 상태와 감정, 그리고 복수라는 테마를 다루며, 복잡한 스토리와 긴장감 넘치는 분위기로 관객들에게 깊은 인상을 남깁니다. 또한, '올드보이'는 한국 영화의 대표적인 작품 중 하나로, 국제적으로도 높은 평가를 받고 있습니다.


In [7]:
# 테스트 실행
try:
    genre = "Drama"
    
    print("영화 추천 시스템")
    print("==================")
    print(f"\n입력 장르: {genre}")
    
    # 1단계 실행 및 결과 출력
    movie_recommendation = recommendation_chain.invoke({"genre": genre})
    print("\n1단계 - 영화 추천 결과:")
    print(movie_recommendation)
    
    print("\n---\n")
    
    # 2단계 실행 및 결과 출력
    plot_summary = plot_chain.invoke({"genre": genre})
    print("2단계 - 영화 줄거리 요약:")
    print(plot_summary)
    
except Exception as e:
    print(f"오류 발생: {e}")

영화 추천 시스템

입력 장르: Drama

1단계 - 영화 추천 결과:
"올드보이"는 2003년에 개봉한 한국 영화입니다. 이 영화는 드라마 장르에 속하며, 강렬한 스토리와 뛰어난 연기로 많은 사람들에게 사랑을 받았습니다. 

올드보이의 줄거리는 다음과 같습니다.

주인공 오대수는 어느 날 갑자기 아무 이유도 모른 채로 15년간 감금당합니다. 그는 감옥에서 탈출하여 자신의 과거와 감금의 진실을 찾기 시작합니다. 그 과정에서 그는 자신이 당한 고통과 복수를 결심하게 됩니다.

올드보이의 추천 이유는 다음과 같습니다.

1.  **강렬한 스토리**: 영화는 강렬한 스토리로 구성되어 있습니다. 감금된 주인공이 탈출하여 자신의 과거를 추적하는 과정이 매우 흥미롭고 짜릿합니다.
2.  **뛰어난 연기**: 주연 배우 최민식, 유해진, 강혜정은 뛰어난 연기로 많은 사람들에게 사랑을 받았습니다. 특히 최민식의 연기는 매우 강렬하고 인상적입니다.
3.  **사회 비판**: 이 영화는 한국 사회의 어두운 면을 비판하는 메시지를 전달합니다. 감금과 복수라는 테마를 통해 사회의 부조리와 인간의 본성을 탐구합니다.

올드보이는 한국 영화의 대표적인 작품 중 하나입니다. 강렬한 스토리와 뛰어난 연기, 사회 비판적인 메시지로 많은 사람들에게 사랑을 받았습니다.

---

2단계 - 영화 줄거리 요약:
Here is a summary of the movie "올드보이" in 10 lines:

*   주인공 오대수(최민식)는 15년간 불법 감금된 채로 살아갑니다. 
*   어느 날 갑자기 석방된 오대수는 자신의 납치와 장기 감금의 이유를 찾기 시작합니다. 
*   오대수는 과거를 조사하면서 자신의 트라우마에 직면하게 됩니다. 
*   그는 과거의 비밀을 풀기 위해 미스터리한 인물들과 대면하게 됩니다. 
*   오대수의 여정에 동참하는 인물은 미스터리한 여성, 이수아(김혜수)입니다. 
*   이수아는 오대수와 복잡한 관계를 형성하며, 그의 과거를 파헤치는 데 도움을 줍니다. 
*   오

## 추가 테스트

In [8]:
# 다른 장르로 추가 테스트
genre2 = "로맨스"

movie_recommendation2 = recommendation_chain.invoke({"genre": genre2})
plot_summary2 = plot_chain.invoke({"genre": genre2})

print(f"다른 장르로 테스트 - {genre2}:")
print("===============================")
print("\n영화 추천:")
print(movie_recommendation2)
print("\n줄거리 요약:")
print(plot_summary2)

다른 장르로 테스트 - 로맨스:

영화 추천:
로맨스 장르의 한국영화로 **"사랑, 소리"**를 추천합니다.

**"사랑, 소리"**는 2016년에 개봉한 한국 영화로, 2020년 7월 14일 넷플릭스에서 스트리밍을 시작했습니다. 이 영화는 로맨스, 드라마, 뮤지컬 장르에 속합니다.

**"사랑, 소리"**는 음악을 통해 서로를 이해하고 사랑을 발견하는 두 남녀의 이야기를 그립니다. 주인공인 소리(김윤아)와 준(정재영)은 서로 다른 세계에서 살지만, 음악을 통해 서로에게 다가갑니다.

**"사랑, 소리"**를 추천하는 이유는 다음과 같습니다:

*   **음악의 힘**: 영화는 음악을 통해 감정을 표현하고, 서로를 이해하는 과정을 아름답게 그립니다. 다양한 음악 장르가 등장하며, 각 음악이 영화의 분위기와 감정을 더욱 깊게 전달합니다.
*   **로맨틱한 분위기**: 영화는 로맨틱한 분위기와 감동적인 장면들로 가득 차 있습니다. 두 주인공이 서로에게 다가가는 과정은 매우 감동적이며, 사랑의 아름다움을 잘 표현합니다.
*   **배우들의 연기**: 김윤아와 정재영은 각각의 캐릭터를 깊이 있게 연기하며, 영화의 감동을 더욱 부각시킵니다.

전체적으로 **"사랑, 소리"**는 로맨스 장르의 영화 중에서도 특히 음악을 통해 사랑과 감정을 표현하는 아름다운 영화입니다.

줄거리 요약:
Here is a summary of the movie "사랑의 불시착" in 10 lines:

* 영화 "사랑의 불시착"은 2019년에 개봉한 한국 로맨스 영화입니다.
* 북한 출신의 여성 윤세리(손예진 분)가 남한에 불시착하게 됩니다.
* 그녀는 남한의 의사 리정혁(현빈 분)을 만나게 됩니다.
* 두 사람은 처음에는 문화 차이와 사회적 차이로 인해 어려움을 겪습니다.
* 하지만 서로를 이해하고 사랑에 빠지게 됩니다.
* 윤세리는 북한 출신이라는 이유로 남한 사회에서 어려움을 겪습니다.
* 리정혁은 윤세리를 도와주며 그녀에 대한 감정을 키웁니다.
* 두 사람은 사랑과 웃음을 통해 서로

## 체인 연결 패턴 설명

In [9]:
# 체인 연결 방식 설명
print("체인 연결 방식 설명:")
print("====================")
print()
print("1단계 체인:")
print("movie_recommendation_prompt | llm | StrOutputParser()")
print()
print("2단계 체인:")
print('{"movie": recommendation_chain} | plot_summary_prompt | llm | StrOutputParser()')
print()
print("핵심 포인트:")
print("- 1단계 체인의 출력이 2단계 체인의 입력으로 자동 전달")
print("- movie 키를 통해 데이터 흐름 제어")
print("- LCEL을 통해 직관적인 파이프라인 구성")

체인 연결 방식 설명:

1단계 체인:
movie_recommendation_prompt | llm | StrOutputParser()

2단계 체인:
{"movie": recommendation_chain} | plot_summary_prompt | llm | StrOutputParser()

핵심 포인트:
- 1단계 체인의 출력이 2단계 체인의 입력으로 자동 전달
- movie 키를 통해 데이터 흐름 제어
- LCEL을 통해 직관적인 파이프라인 구성


## 학습 정리

### 문제 2에서 배운 핵심 개념
1. **ChatPromptTemplate**: 시스템과 사용자 메시지 구조화
2. **다단계 체인**: 여러 단계로 나누어 복잡한 작업 처리
3. **체인 간 데이터 전달**: {"key": previous_chain} 패턴
4. **단계별 결과 확인**: 각 체인의 중간 결과 출력

### 구현 완료 사항
- ✅ 2단계 체인 구현
- ✅ ChatPromptTemplate 사용
- ✅ 체인 간 데이터 전달
- ✅ 영화 추천 기능
- ✅ 줄거리 요약 기능
- ✅ 각 단계별 결과 출력
- ✅ 다양한 장르 테스트

### 체인 연결의 핵심 패턴
```python
# 기본 패턴
chain1 = prompt1 | llm | parser
chain2 = {"input_key": chain1} | prompt2 | llm | parser
```

이 패턴을 통해 복잡한 작업을 단계별로 분해하여 처리할 수 있습니다.